In [1]:
import sys
sys.path.append("../..")

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
# environments

from environment.qlearning.obstacle_avoidance_env import ObstacleAvoidanceEnv

In [4]:
from utils.reader import get_yaml_path, read_file

In [5]:
from agent.qagent import QAgent

In [6]:
from evaluation.agent_evaluation import evaluate

In [7]:
server_address = "localhost:50060" # adjust the port as needed
client_name = "RLClient"
env = ObstacleAvoidanceEnv(server_address, client_name) # adjust the env as needed
env.connect_to_client()

2025-11-11 18:18:36,610 — INFO — ✓ Connected to localhost:50060



In [8]:
agent = QAgent(env)
agent_id = "00000000-0000-0000-0000-000000000001"
agents = { agent_id: agent }

In [9]:
configs_path = get_yaml_path("src", "scripts", "resources", "generated", "q-obstacle-avoidance-test")
yml_files = list(configs_path.glob("*.yml"))
configs = [read_file(f) for f in yml_files]

In [10]:
agent_path = get_yaml_path("src", "scripts", "resources", "generated", "checkpoints", "obstacle-avoidance-big-v10", "oa_ep884")
for agent in agents.values():
    agent.load(agent_path)

2025-11-11 18:18:36,624 — INFO — Agent loaded from /home/simone/uni-lab/PPS/2024/SRS/PPS-22-srs/feature/experimental-evaluation/python/src/scripts/resources/generated/checkpoints/obstacle-avoidance-big-v10/oa_ep884.npz
2025-11-11 18:18:36,625 — INFO —   Q-table shape: (64, 5)
2025-11-11 18:18:36,625 — INFO —   Current epsilon: 0.1318
2025-11-11 18:18:36,625 — INFO —   Total episodes trained: 884
0.01


In [11]:
did_succeed=lambda reward, termination, truncation: True if truncation and reward > -500 else False

In [12]:
success_rate, median_steps_to_success = evaluate(env=env, agents=agents, configs=configs, max_steps=5000, did_succeed=did_succeed)

Evaluation: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:35<00:00,  3.59s/configuration run]


In [13]:
print(f"success_rate:             {success_rate}")
print(f"median steps to success:  {median_steps_to_success}")

success_rate:             {'00000000-0000-0000-0000-000000000001': 0.5}
median steps to success:  {'00000000-0000-0000-0000-000000000001': np.float64(5000.0)}
